In [ ]:
import pandas as pd

# Cargar CSV en un DataFrame
df = pd.read_csv("raw_data/athlete_events.csv")

In [2]:
df.info()

#Vemos que existen muchos nulos en medal (los que no ganaron medalla), pero tambien hay nulos para edad, height y weight

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


In [3]:
df.describe()

#vemos que Year es una columna numérica cuando debería ser categórica. 

,ID,Age,Height,Weight,Year
count,271116.000000,261642.000000,210945.000000,208241.000000,271116.000000
mean,68248.954396,25.556898,175.338970,70.702393,1978.378480
std,39022.286345,6.393561,10.518462,14.348020,29.877632
min,1.000000,10.000000,127.000000,25.000000,1896.000000
25%,34643.000000,21.000000,168.000000,60.000000,1960.000000
50%,68205.000000,24.000000,175.000000,70.000000,1988.000000
75%,102097.250000,28.000000,183.000000,79.000000,2002.000000
max,135571.000000,97.000000,226.000000,214.000000,2016.000000


In [4]:
#Vamos a cambiar los NaN por la etiqueta 'No medalla' 

df['Medal'] = df['Medal'].fillna('No medal')
df

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No medal
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No medal
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,No medal
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,No medal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,No medal
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",No medal
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",No medal
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,No medal


In [5]:
df.Medal.value_counts(normalize=True)

#Vemos que las clases del target estan muy desbalanceadas, habra que aplicar algÚn metodo de balanceo para equilibrarlas. 

Medal
No medal    0.853262
Gold        0.049322
Bronze      0.049038
Silver      0.048378
Name: proportion, dtype: float64

In [6]:
#Para eliminar los nulos de las columnas Age, Height y Weight, vamos a crear una media agrupando por pais y por deporte.
columnas_nulos = ['Age', 'Height', 'Weight']

# Imputar usando la media por genero, Team (Mejor NOC) y Sport
for col in columnas_nulos:
    df[col] = df.groupby(['Sex', 'NOC', 'Sport'])[col].transform(
        lambda x: x.fillna(x.mean())
    )
#Para los valores donde no se puede agrupar por sex, NOC y Sport, eliminamos el pais
for col in columnas_nulos:
    df[col] = df.groupby(['Sex', 'Sport'])[col].transform(
        lambda x: x.fillna(x.mean())
    )
#Para los valores donde no se puede agrupar por sex y Sport, eliminamos el deporte
for col in columnas_nulos:
    df[col] = df.groupby(['Sex'])[col].transform(
        lambda x: x.fillna(x.mean())
    )


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     271116 non-null  float64
 4   Height  271116 non-null  float64
 5   Weight  271116 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   271116 non-null  object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


In [8]:
#Vemos que a veces hay diferentes formas de llamar a un pais, y puede ser complicado con los paises que ya no existen o se han fusionado.

paises_unicos = df['Team'].unique()
df_paises = pd.DataFrame(sorted(df['Team'].unique()), columns=['Pais'])
print(df_paises)

                       Pais
0               30. Februar
1     A North American Team
2                 Acipactli
3                   Acturus
4               Afghanistan
...                     ...
1179                 Zambia
1180                Zefyros
1181               Zimbabwe
1182                    Zut
1183                   rn-2

[1184 rows x 1 columns]


In [9]:
#Vemos que hay muchos paises con nombres raros, equipos que no se sabe bien a donde pertenecen... como los datos que tenemos son de TODOS
#los JJOO de la historia, el primero data de 1896 hasta 2016. Vamos a intentar reducir los años y eliminar los primeros juegos olimpicos
#que no son nada representativos en cuanto a deporte, equipos etc para hacer predicciones en la actualidad.
#Vamos a quedarnos con los JJOO de mas alla de 1970 para evitar esto. 

df = df[df['Year'] >= 1970]

print(df['Year'].min())

1972


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 179878 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      179878 non-null  int64  
 1   Name    179878 non-null  object 
 2   Sex     179878 non-null  object 
 3   Age     179878 non-null  float64
 4   Height  179878 non-null  float64
 5   Weight  179878 non-null  float64
 6   Team    179878 non-null  object 
 7   NOC     179878 non-null  object 
 8   Games   179878 non-null  object 
 9   Year    179878 non-null  int64  
 10  Season  179878 non-null  object 
 11  City    179878 non-null  object 
 12  Sport   179878 non-null  object 
 13  Event   179878 non-null  object 
 14  Medal   179878 non-null  object 
dtypes: float64(3), int64(2), object(10)
memory usage: 22.0+ MB


In [11]:
#Una vez que hemos utlizado el año para filtrar, cambiamos el año y el ID a categorica
df['Year'] = df['Year'].astype(str)
df['ID'] = df['ID'].astype(str)

C:\Users\Alvar\AppData\Local\Temp\ipykernel_26092\3827622361.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year'] = df['Year'].astype(str)
C:\Users\Alvar\AppData\Local\Temp\ipykernel_26092\3827622361.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ID'] = df['ID'].astype(str)


In [12]:
df.Medal.value_counts(normalize=True) #vemos que la proporcion no ha cambiado mucho eliminando JJOO de antes de 1970

Medal
No medal    0.861734
Bronze      0.047315
Gold        0.045720
Silver      0.045231
Name: proportion, dtype: float64

In [13]:
df.Medal.value_counts() #y que aún tenemos muchos valores absolutos de medallas. 

Medal
No medal    155007
Bronze        8511
Gold          8224
Silver        8136
Name: count, dtype: int64

In [14]:
#Vemos que pasamos de 1183 equipos o paises a 386

paises_unicos = df['Team'].unique()
df_paises = pd.DataFrame(sorted(df['Team'].unique()), columns=['Pais'])
print(df_paises)

               Pais
0       Afghanistan
1           Albania
2           Algeria
3    American Samoa
4           Andorra
..              ...
381      Yugoslavia
382    Yugoslavia-1
383    Yugoslavia-2
384          Zambia
385        Zimbabwe

[386 rows x 1 columns]


In [15]:
#Vamos a intentar asignar los paises de la época a paises actuales. Para ello lo primero eliminamos de nuestro df_paises los paises que existen actualmente.

paises_actuales = [
    "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda",
    "Argentina", "Armenia", "Australia", "Austria", "Azerbaijan", "Bahamas", "Bahrain",
    "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize", "Benin", "Bhutan", "Bolivia",
    "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei", "Bulgaria", "Burkina Faso",
    "Burundi", "Cabo Verde", "Cambodia", "Cameroon", "Canada", "Central African Republic",
    "Chad", "Chile", "China", "Colombia", "Comoros", "Congo (Brazzaville)", "Congo (Kinshasa)",
    "Costa Rica", "Croatia", "Cuba", "Cyprus", "Czech Republic", "Denmark", "Djibouti",
    "Dominica", "Dominican Republic", "Ecuador", "Egypt", "El Salvador", "Equatorial Guinea",
    "Eritrea", "Estonia", "Eswatini", "Ethiopia", "Fiji", "Finland", "France", "Gabon",
    "Gambia", "Georgia", "Germany", "Ghana", "Greece", "Grenada", "Guatemala", "Guinea",
    "Guinea-Bissau", "Guyana", "Haiti", "Honduras", "Hungary", "Iceland", "India", "Indonesia",
    "Iran", "Iraq", "Ireland", "Israel", "Italy", "Ivory Coast", "Jamaica", "Japan", "Jordan",
    "Kazakhstan", "Kenya", "Kiribati", "Kuwait", "Kyrgyzstan", "Laos", "Latvia", "Lebanon",
    "Lesotho", "Liberia", "Libya", "Liechtenstein", "Lithuania", "Luxembourg", "Madagascar",
    "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands", "Mauritania",
    "Mauritius", "Mexico", "Micronesia", "Moldova", "Monaco", "Mongolia", "Montenegro",
    "Morocco", "Mozambique", "Myanmar", "Namibia", "Nauru", "Nepal", "Netherlands", "New Zealand",
    "Nicaragua", "Niger", "Nigeria", "North Korea", "North Macedonia", "Norway", "Oman", "Pakistan",
    "Palau", "Palestine", "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines", "Poland",
    "Portugal", "Puerto Rico", "Qatar", "Romania", "Russia", "Rwanda", "Saint Kitts and Nevis",
    "Saint Lucia", "Saint Vincent and the Grenadines", "Samoa", "San Marino", "Sao Tome and Principe",
    "Saudi Arabia", "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Slovakia",
    "Slovenia", "Solomon Islands", "Somalia", "South Africa", "South Korea", "South Sudan", "Spain",
    "Sri Lanka", "Sudan", "Suriname", "Sweden", "Switzerland", "Syria", "Tajikistan", "Tanzania",
    "Thailand", "Timor-Leste", "Togo", "Tonga", "Trinidad and Tobago", "Tunisia", "Turkey",
    "Turkmenistan", "Tuvalu", "Uganda", "Ukraine", "United Arab Emirates", "United Kingdom",
    "United States", "Uruguay", "Uzbekistan", "Vanuatu", "Venezuela", "Vietnam", "Yemen",
    "Zambia", "Zimbabwe"
]

# Suponiendo que la columna con los países se llama 'Team'
df_not_current = df_paises[~df_paises['Pais'].isin(paises_actuales)]

# Eliminar duplicados si quieres solo la lista única de países
df_not_current = df_not_current.drop_duplicates(subset=['Pais'])

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

df_not_current

,Pais
3,American Samoa
8,Argentina-1
9,Argentina-2
11,Aruba
13,Australia-1
14,Australia-2
15,Australia-3
17,Austria-1
18,Austria-2
25,Belarus-1


In [16]:
#vamos ahora a crear un diccionario y asignar los paises actuales a los nombres de paises antiguos. 

asignacion_paises = {
    "American Samoa": "Samoa",
    "Argentina-1": "Argentina",
    "Argentina-2": "Argentina",
    "Aruba": "Aruba",
    "Australia-1": "Australia",
    "Australia-2": "Australia",
    "Australia-3": "Australia",
    "Austria-1": "Austria",
    "Austria-2": "Austria",
    "Belarus-1": "Belarus",
    "Belarus-2": "Belarus",
    "Bermuda": "Bermuda",
    "Bonaparte": "France",
    "Brazil-1": "Brazil",
    "Brazil-2": "Brazil",
    "Brentina": "Italy",
    "Briar": "United Kingdom",
    "British Virgin Islands": "Virgin Islands",
    "Bulgaria-1": "Bulgaria",
    "Bulgaria-2": "Bulgaria",
    "Calimucho": "Spain",
    "Canada-1": "Canada",
    "Canada-2": "Canada",
    "Canada-3": "Canada",
    "Cape Verde": "Cape Verde",
    "Cayman Islands": "Cayman Islands",
    "China-1": "China",
    "China-2": "China",
    "China-3": "China",
    "Chinese Taipei": "Taiwan",
    "Chinese Taipei-1": "Taiwan",
    "Chinese Taipei-2": "Taiwan",
    "Clearwater": "United States",
    "Cook Islands": "Cook Islands",
    "Congo (Brazzaville)": "Congo",
    "Congo (Kinshasa)": "Congo",
    "Cote d'Ivoire": "Ivory Coast",
    "Cuba-1": "Cuba",
    "Cuba-2": "Cuba",
    "Czech Republic-1": "Czech Republic",
    "Czech Republic-2": "Czech Republic",
    "Czechoslovakia": "Czech Republic",
    "Czechoslovakia-1": "Czech Republic",
    "Czechoslovakia-2": "Czech Republic",
    "Czechoslovakia-3": "Czech Republic",
    "Denmark-1": "Denmark",
    "Denmark-2": "Denmark",
    "Diabolo St Maurice": "Switzerland",
    "Digby": "United Kingdom",
    "Don Schufro": "Germany",
    "Dow Jones": "United States",
    "East Germany": "Germany",
    "East Germany-1": "Germany",
    "East Germany-2": "Germany",
    "East Germany-3": "Germany",
    "Elvis Va": "United States",
    "Federated States of Micronesia": "Micronesia",
    "Floresco": "France",
    "France-1": "France",
    "France-2": "France",
    "France-3": "France",
    "Galopin De La Font": "France",
    "Germany-1": "Germany",
    "Germany-2": "Germany",
    "Germany-3": "Germany",
    "Gran Gesto": "Spain",
    "Great Britain": "United Kingdom",
    "Great Britain-1": "United Kingdom",
    "Great Britain-2": "United Kingdom",
    "Great Britain-3": "United Kingdom",
    "Greece-1": "Greece",
    "Greece-2": "Greece",
    "Greenoaks Dundee": "United Kingdom",
    "Guam": "Guam",
    "Guinea Bissau": "Guinea-Bissau",
    "Hong Kong": "Hong Kong",
    "Hong Kong-1": "Hong Kong",
    "Hong Kong-2": "Hong Kong",
    "Hungary-1": "Hungary",
    "Hungary-2": "Hungary",
    "India-1": "India",
    "India-2": "India",
    "Individual Olympic Athletes": "Russia",
    "Indonesia-1": "Indonesia",
    "Indonesia-2": "Indonesia",
    "Ireland-1": "Ireland",
    "Ireland-2": "Ireland",
    "Israel-1": "Israel",
    "Israel-2": "Israel",
    "Italy-1": "Italy",
    "Italy-2": "Italy",
    "Italy-3": "Italy",
    "Jamaica-1": "Jamaica",
    "Jamaica-2": "Jamaica",
    "Japan-1": "Japan",
    "Japan-2": "Japan",
    "Japan-3": "Japan",
    "Kosovo": "Kosovo",
    "Lancet": "United Kingdom",
    "Latvia-1": "Latvia",
    "Latvia-2": "Latvia",
    "Lucky Star": "United Kingdom",
    "Macedonia": "North Macedonia",
    "Malaysia-1": "Malaysia",
    "Malaysia-2": "Malaysia",
    "Mauritius-1": "Mauritius",
    "Mauritius-2": "Mauritius",
    "Mexico-1": "Mexico",
    "Mexico-2": "Mexico",
    "Mistral Hojris": "Denmark",
    "Monaco-1": "Monaco",
    "Monaco-2": "Monaco",
    "Mythilus": "Greece",
    "Nadine": "United Kingdom",
    "Netherlands Antilles": "Curaçao",
    "Netherlands-1": "Netherlands",
    "Netherlands-2": "Netherlands",
    "New Zealand-1": "New Zealand",
    "New Zealand-2": "New Zealand",
    "Nigeria-1": "Nigeria",
    "Nigeria-2": "Nigeria",
    "North Korea-1": "North Korea",
    "North Korea-2": "North Korea",
    "North Yemen": "Yemen",
    "Norway-1": "Norway",
    "Norway-2": "Norway",
    "Notavel": "Portugal",
    "Orion": "United Kingdom",
    "Oxalis": "United Kingdom",
    "Poland-1": "Poland",
    "Poland-2": "Poland",
    "Poland-3": "Poland",
    "Pop Art": "United States",
    "Portugal-1": "Portugal",
    "Portugal-2": "Portugal",
    "Puerto Rico-1": "Puerto Rico",
    "Puerto Rico-2": "Puerto Rico",
    "Quando Quando": "Italy",
    "Rambo": "United States",
    "Ravel": "France",
    "Refugee Olympic Athletes": "Refugees",
    "Relampago": "Spain",
    "Romania-1": "Romania",
    "Romania-2": "Romania",
    "Russia-1": "Russia",
    "Russia-2": "Russia",
    "Russia-3": "Russia",
    "Salinero": "Spain",
    "Satchmo": "United States",
    "Serbia and Montenegro": "Serbia",
    "Serbia-1": "Serbia",
    "Serbia-2": "Serbia",
    "Singapore-1": "Singapore",
    "Singapore-2": "Singapore",
    "Slovakia-1": "Slovakia",
    "Slovakia-2": "Slovakia",
    "Slovenia-1": "Slovenia",
    "Slovenia-2": "Slovenia",
    "Solos Carex": "France",
    "South Korea-1": "South Korea",
    "South Korea-2": "South Korea",
    "South Vietnam": "Vietnam",
    "South Yemen": "Yemen",
    "Soviet Union": "Russia",
    "Soviet Union-1": "Russia",
    "Soviet Union-2": "Russia",
    "Soviet Union-3": "Russia",
    "Spain-1": "Spain",
    "Spain-2": "Spain",
    "Sunrise": "USA",
    "Swaziland": "Eswatini",
    "Sweden-1": "Sweden",
    "Sweden-2": "Sweden",
    "Switzerland-1": "Switzerland",
    "Switzerland-2": "Switzerland",
    "Thailand-1": "Thailand",
    "Thailand-2": "Thailand",
    "Timor Leste": "East Timor",
    "Ukraine-1": "Ukraine",
    "Ukraine-2": "Ukraine",
    "Unified Team": "Russia",
    "Unified Team-1": "Russia",
    "Unified Team-2": "Russia",
    "Unified Team-3": "Russia",
    "United States Virgin Islands": "Virgin Islands",
    "United States Virgin Islands-1": "Virgin Islands",
    "United States Virgin Islands-2": "Virgin Islands",
    "United States-1": "United States",
    "United States-2": "United States",
    "United States-3": "United States",
    "Uzbekistan-1": "Uzbekistan",
    "Uzbekistan-2": "Uzbekistan",
    "West Germany": "Germany",
    "West Germany-1": "Germany",
    "West Germany-2": "Germany",
    "West Germany-3": "Germany",
    "Whisper": "United Kingdom",
    "Whitini Star": "United Kingdom",
    "Yugoslavia": "Serbia",
    "Yugoslavia-1": "Serbia",
    "Yugoslavia-2": "Serbia"
}

# Aplicamos el diccionario 
df['Team'] = df['Team'].replace(asignacion_paises)

#Vemos que ahora tenemos 206 paises que son los que compiten actualmente en los JJOO
df['Team'].nunique()

C:\Users\Alvar\AppData\Local\Temp\ipykernel_26092\2901044255.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Team'] = df['Team'].replace(asignacion_paises)


206

In [17]:
df.head(20)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No medal
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No medal
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,No medal
5,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",No medal
6,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,No medal
7,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,"Speed Skating Women's 1,000 metres",No medal
8,5,Christine Jacoba Aaftink,F,27.0,185.0,82.0,Netherlands,NED,1994 Winter,1994,Winter,Lillehammer,Speed Skating,Speed Skating Women's 500 metres,No medal
9,5,Christine Jacoba Aaftink,F,27.0,185.0,82.0,Netherlands,NED,1994 Winter,1994,Winter,Lillehammer,Speed Skating,"Speed Skating Women's 1,000 metres",No medal
10,6,Per Knut Aaland,M,31.0,188.0,75.0,United States,USA,1992 Winter,1992,Winter,Albertville,Cross Country Skiing,Cross Country Skiing Men's 10 kilometres,No medal
11,6,Per Knut Aaland,M,31.0,188.0,75.0,United States,USA,1992 Winter,1992,Winter,Albertville,Cross Country Skiing,Cross Country Skiing Men's 50 kilometres,No medal


In [18]:
df.Sport.value_counts()

Sport
Athletics                    24100
Swimming                     17410
Gymnastics                   13453
Cross Country Skiing          7209
Rowing                        6682
Cycling                       6560
Alpine Skiing                 6432
Shooting                      6217
Canoeing                      4978
Fencing                       4776
Biathlon                      4696
Sailing                       4272
Wrestling                     4227
Football                      4128
Ice Hockey                    3902
Speed Skating                 3893
Boxing                        3804
Equestrianism                 3757
Judo                          3729
Hockey                        3722
Handball                      3560
Volleyball                    3019
Basketball                    3015
Weightlifting                 2687
Water Polo                    2310
Bobsleigh                     2173
Archery                       2069
Tennis                        2024
Table Tennis  

In [ ]:
#Vemos que existen 51 disciplinas de deportes
df.Sport.nunique()

51

In [ ]:
#Tambien podemos intentar reducir los datos eliminando las disciplinar que actualmente no estan en los JJOO y por tanto no necesitamos predecir. 

sports_actual = [
    "Athletics",
    "Swimming",
    "Gymnastics",
    "Rowing",
    "Cycling",
    "Sailing",
    "Shooting",
    "Wrestling",
    "Football", 
    "Equestrianism",
    "Judo",
    "Handball",
    "Volleyball",
    "Basketball",
    "Weightlifting",
    "Water Polo",
    "Archery",
    "Tennis",
    "Table Tennis",
    "Taekwondo",
    "Triathlon",
    "Badminton",
    "Cross Country Skiing",
    "Alpine Skiing",
    "Biathlon",
    "Bobsleigh",
    "Ice Hockey",
    "Speed Skating",
    "Ski Jumping",
    "Figure Skating",
    "Short Track Speed Skating",
    "Nordic Combined",
    "Freestyle Skiing",
    "Snowboarding",
    "Luge",
    "Modern Pentathlon",
    "Rugby Sevens",
    "Curling",
    "Golf",
    "Synchronized Swimming",
    "Boxing",
    "Rhythmic Gymnastics",
    "Canoeing",
    "Skeleton",
    "Beach Volleyball",
    "Trampolining",
    "Fencing",
    "Hockey",
    "Diving"
]

deportes_originales = set(df["Sport"].unique())

df = df[df["Sport"].isin(sports_actual)]
deportes_eliminados = deportes_originales - set(df["Sport"].unique())

print("Deportes eliminados:")
print(deportes_eliminados)
df.Sport.nunique()

#Despues del filtrado, eliminamos Softball y Baseball que ya no son olímpicos.

Deportes eliminados:
{'Softball', 'Baseball'}


49

In [ ]:
df.to_csv("processed_data/athlete_events_clean.csv", index=False)